<h1> Exercice </h1>

Le dataset "Titres Netflix" est une compilation exhaustive de films et de séries télévisées disponibles sur Netflix, couvrant divers aspects tels que le type de titre, le réalisateur, les acteurs, le pays de production, l'année de sortie, la classification, la durée, les genres (répertoriés dans) et une brève description. Ce dataset est essentiel pour analyser les tendances du contenu Netflix, comprendre la popularité des genres et examiner la distribution du contenu à travers différentes régions et périodes.

Les colonnes : 
- show_id : Un identifiant unique pour chaque titre.
- type : La catégorie du titre, qui peut être 'Film' ou 'Série télévisée'.
- title : Le nom du film ou de la série télévisée.
- director : Le(s) réalisateur(s) du film ou de la série télévisée. (Contient des valeurs nulles pour certaines entrées, en particulier les séries télévisées où cette information peut ne pas être applicable.)
- cast : La liste des acteurs principaux du titre. (Certaines entrées peuvent ne pas avoir cette information.)
- country : Le pays ou les pays où le film ou la série télévisée a été produit.
- date_added : La date à laquelle le titre a été ajouté à Netflix.
- release_year : L'année de sortie originale du film ou de la série télévisée.
- rating : La classification par âge du titre.
- duration : La durée du titre, en minutes pour les films et en saisons pour les séries télévisées.
- listed_in : Les genres auxquels appartient le titre.
- description : Un bref résumé du titre.

<h1> Enoncé </h1>

<h2> data cleaning & data modeling </h2>

<b> 1) Importation du fichier </b>

Tentez d'importer le fichier netflix_titles.csv. Si vous rencontrez une erreur, celle-ci est probablement liée à un problème d'encodage. Trouvez l'encodage adéquat pour lire le fichier correctement.

<b> 2) Création d'une copie du DataFrame </b>

Créez une copie de votre DataFrame dans une nouvelle variable afin de conserver les données originales accessibles.

<b> 3) Suppression des colonnes inutiles </b>

Vérifiez que les colonnes nommées "Unnamed:..." en fin de DataFrame ne contiennent aucune donnée, puis supprimez-les.

<b> 4) Définition d'un nouvel index </b>

Assurez-vous que la colonne "Show_id" ne possède ni valeur nulle ni doublon. Si c'est le cas, définissez show_id comme index du DataFrame.

<b> 5) Correction du type de colonne </b>
Vérifiez le type de la colonne "Date_added" et corrigez-le avec le type adéquat si nécessaire.

<b> 6) Gestion de la colonne "Duration" </b>

- Confirmez que la colonne "Type" contient uniquement les valeurs 'Movie' et 'TV Show'.
- Examinez la nomenclature des valeurs de la colonne "Duration".
- Via un groupby, vérifiez que les durées sont bien associées à une durée en minutes pour les films et à un nombre de saisons pour les séries.
- Créez une nouvelle colonne 'duration (movies)' pour isoler le nombre de minutes pour les films. Assurez-vous que cette  colonne soit de type "float".
- Créez une colonne 'seasons (TV Show)' pour isoler le nombre de saisons pour les séries. Assurez-vous que cette colonne soit de type "float".
- Supprimez finalement la colonne 'Duration' du dataset.

<b> 7) Création de DataFrames annexes pour les valeurs séparées par des virgules </b>

Certaines colonnes comme country, cast et listed_in contiennent pour certains titres des séries de valeurs, sépérées par des virgules. Nous souhaitons créer des tableaux annexes permettant d'avoir chaque valeur de manière distincte.

Pour la colonne "Country" :
- Créez une colonne 'countries' en transformant les valeurs de la colonne 'country' en listes (via str.split).
- Créez un DataFrame 'countries_exploded' qui génère une ligne pour chaque pays via la méthode .explode().
- Isolez uniquement la colonne 'countries' dans le DataFrame 'countries_exploded' (non vue en cours, à vous de comprendre son fonctionnement : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html)
- Répétez ce processus pour les colonnes 'categories' (basée sur 'listed_in') et 'cast', créant respectivement les - DataFrames 'categories_exploded' et 'cast_list'.

<b> 8) Suppression des colonnes transformées </b>

Supprimez les trois colonnes transformées du DataFrame original.

<b> 9) Création de colonnes temporelles </b>

À partir de 'Date Added', créez des colonnes pour l'année d'ajout, le mois d'ajout et le jour de la semaine d'ajout.
Analyse des Données

<h2> Analyse des données </h2>

Tentez maintenant de répondre aux questions suivantes : 

1) Combien de "shows" sont présents dans ce dataset ?
2) Quelle est la répartition entre les types 'Movie' et 'TV Show' ?
3) Quelle est la répartition des ajouts en fonction de l'année ?
4) Quel est le top 5 des catégories de shows les plus ajoutées ?
5) Quel est le top 5 des comédiens les plus plébiscités aux États-Unis ?
6) Quelle est la répartition des ajouts en fonction du jour de la semaine ?
7) Dans quel pays sont produits le plus de documentaires ?
8) En moyenne, combien de saisons ont les séries ?
9) Quelle est la distribution des films en fonction de leur durée (quartiles) ?
10) Combien de shows ont pour thématique la drogue (présence du mot "drug" dans la description) ?

<h1> Data Cleaning et data modeling</h1>

<h1> Analyse des données </h1>

In [96]:
import pandas as pd
#Importation du fichier netflix_titles.csv. 
df = pd.read_csv(
    r"C:\Users\cblanchon\OneDrive - Elior-Group\Bureau\python\notebok\fichiers_cours_data_science\fichiers_cours_data_science\2_fichier_cours_data_analyse\2_Pandas\8_ateliers\1_Netflix\netflix_titles.csv",
    encoding="latin1"
)

#Copie du DataFrame
dfcopie = df

#clean colonne
df.drop(df.filter(like="Unnamed").columns, axis=1, inplace=True)

#index

print(df["show_id"].isnull().any())
print(df["show_id"].duplicated().any())

df.set_index('show_id')

#Correction du type de colonne
df['date_added']
df['date_added'] = pd.to_datetime(df['date_added'], format='%B %d, %Y', errors='coerce')

#Gestion de la colonne 

df
df['type'].unique()
df['duration'].unique()
df
mask_incorrect = ~(
    ((df['type'] == 'Movie') & df['duration'].str.contains('min', na=False)) |
    ((df['type'] == 'TV Show') & df['duration'].str.contains('Season', na=False))
)
df.loc[mask_incorrect, 'duration'] = df.loc[mask_incorrect, 'rating']

df['duration (movies)'] = df['duration'].apply(
    lambda x: float(x.split()[0]) if 'min' in str(x) else None
)

df['seasons (TV Show)'] = df['duration'].apply(
    lambda x: float(x.split()[0]) if 'Seasons' in str(x) else None
)
df.drop('duration', axis=1, inplace=True)


#country 

df['countries'] = df['country'].str.split(', ')
countries_exploded = df.explode('countries')
countries_exploded = countries_exploded['countries']
countries_exploded.to_csv('countries_exploded.csv')
countries_exploded

#Liste

df['listed_inLIST'] = df['listed_in'].str.split(', ')
listed_in_exploded = df.explode('listed_inLIST')
listed_in_exploded = listed_in_exploded['listed_inLIST']
listed_in_exploded.to_csv('listed_inLISTs_exploded.csv', index=False)
listed_in_exploded.unique()

#cast 

df['cast_list'] = df['cast'].str.split(', ')
cast_exploded = df.explode('cast_list')
cast_exploded = cast_exploded['cast_list']
cast_exploded.to_csv('cast_list_exploded.csv', index=False)
cast_exploded

#clean tableau
df.drop(columns=['countries','listed_inLIST','cast_list'],inplace=True)

#création de colonnes temporelles
df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month
df['day_of_week'] = df['date_added'].dt.day_name()

df[['date_added', 'year', 'month', 'day_of_week']].head()

#ANALYSE DE LA DONNEE


#Combien de "shows"
df['type'].value_counts()
# Répartition entre 'Movie' et 'TV Show'
df['type'].value_counts(normalize=True) * 100
# Répartition des ajouts en fonction de l'année
df['year'].value_counts(normalize=True) * 100
# Top 5 des catégories de shows 
(listed_in_exploded.value_counts(normalize=True) * 100).head(5)
# Top 5 des comédiens
(cast_exploded.value_counts(normalize=True)*100).head(5)
# Répartition des ajouts en fonction du jour de la semaine 
(df['day_of_week'].value_counts(normalize=True)*100)
# Moyenne, du nmbre de saisons par series
df['seasons (TV Show)'].mean()
# Distribution des films en fonction de leur durée
df['duration (movies)'].describe()
# Shows qui ont pour thématique la drogue 
drug=df[df['description'].str.contains('drug')]
print(len('drug'))

df

False
False
4


,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration (movies),seasons (TV Show),year,month,day_of_week
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,Documentaries,"As her father nears the end of his life, filmm...",90.0,NaN,2021.0,9.0,Saturday
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,2.0,2021.0,9.0,Friday
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,NaN,2021.0,9.0,Friday
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN,NaN,2021.0,9.0,Friday
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN,2.0,2021.0,9.0,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009,R,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,88.0,NaN,2019.0,11.0,Friday
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006,PG,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",88.0,NaN,2020.0,1.0,Saturday
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,2019-03-02,2015,TV-14,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...,111.0,NaN,2019.0,3.0,Saturday
8807,s8808,TV Show,Parasyte: The Grey,Yeon Sang-ho,"Shin Hyun-been, Jeon Yeo-bin, Goo Kyo-hwan",South Korea,2024-04-05,2024,TV-MA,"Sci-fi, Horror, Action",A new breed of parasitic aliens arrive on Eart...,NaN,NaN,2024.0,4.0,Friday
